# Predict Bike Sharing Demand with Autogluon

In [ ]:
import pandas as pd
from autogluon.tabular import TabularPredictor
import autogluon
import matplotlib.pyplot as plt
print('Autogluon Version: ', autogluon.common.version.__version__)

## Step-1: Load and explore the data

In [ ]:
train = pd.read_csv('bike-sharing-demand/train.csv')
train['datetime'] = pd.to_datetime(train['datetime'])
print(train.info())
train.head()

In [ ]:
test = pd.read_csv('bike-sharing-demand/test.csv')
test['datetime'] = pd.to_datetime(test['datetime'])
print(test.info())
test.head()

In [ ]:
submission = pd.read_csv('bike-sharing-demand/sampleSubmission.csv')
print(submission.info())
submission.tail()

## Step-2: Train Model using Autogluon's Tabular Prediction

In [ ]:
predictor = TabularPredictor(label='count', problem_type='regression')
predictor = predictor.fit(
    train_data=train.drop(columns=['casual', 'registered']),
    time_limit=300,
    presets='high_quality'
)

In [ ]:
predictor.fit_summary()

In [ ]:
predictions = predictor.predict(test)

### Checking for any negative predictions and set them to 0

In [ ]:
print(f'Number of negative predictions: {(predictions < 0).sum()}')
predictions[predictions < 0] = 0

In [ ]:
submission['count'] = predictions
submission.to_csv('bike-sharing-demand/submission.csv', index=False)

## Feature Engineering with EDA

In [ ]:
train.hist(figsize=(15,12))

In [ ]:
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour